In [ ]:
from collections import defaultdict

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy.linalg import orthogonal_procrustes
from scipy.stats import ortho_group
from tqdm import tqdm

In [ ]:
rng = np.random.default_rng(1234)
N = 1000
D = 10
n_repetitions = 10
n_permutations = 10

results = {"D": [], "dist": [], "value": [], "noise": []}

for i in [1, 5, 10, 20, 30, 100, 200, 300]:
# for i in [1, 5, 10, 20, 30]:
    increased_D = i * D
    for noise_level in [1e-2, 5e-2, 1e-1]:
        for repetition in tqdm(range(n_repetitions)):
            # One random matrix and one rotated version with added noise
            Q = ortho_group.rvs(increased_D, size=1)
            a = rng.standard_normal((N, increased_D))
            b = a @ Q + noise_level * rng.standard_normal((N, increased_D))

            r, scale = orthogonal_procrustes(a, b)
            total_norm = (
                -2 * scale
                + np.linalg.norm(a, ord="fro") ** 2
                + np.linalg.norm(b, ord="fro") ** 2
            )
            
            results["D"].append(increased_D)
            results["dist"].append("ortho_proc")
            results["value"].append(np.sqrt(total_norm))
            results["noise"].append(noise_level)
            
            results["D"].append(increased_D)
            results["dist"].append("squared_ortho_proc")
            results["value"].append(total_norm)
            results["noise"].append(noise_level)
            
            # baseline value for unrelated matrices
            for _ in range(n_permutations):
                a_shuffled = rng.permutation(a, axis=0)
                shuffled_norm = (
                    -2 * orthogonal_procrustes(a_shuffled, b)[1]
                    + np.linalg.norm(a_shuffled, ord="fro") ** 2
                    + np.linalg.norm(b, ord="fro") ** 2
                )
                results["D"].append(increased_D)
                results["dist"].append("shuffled")
                results["value"].append(np.sqrt(shuffled_norm))
                results["noise"].append(noise_level)
                
                results["D"].append(increased_D)
                results["dist"].append("squared_shuffled")
                results["value"].append(shuffled_norm)
                results["noise"].append(noise_level)
    
df = pd.DataFrame.from_dict(results)


In [ ]:
# df.to_parquet("procrustes_dim_increase.parquet")

In [ ]:
df = pd.read_parquet("procrustes_dim_increase.parquet")
df.head()

Procrustes distance and dimensionality are in a powerlaw relation (linear in the log-log plot).
If you decrease N to something smaller like 100, the curve will flatten at that dimension.
With larger N,

In [ ]:
df["Representations"] = df["dist"]
df["Noise SD"] = df["noise"]
df.loc[df["Representations"] == "ortho_proc", "Representations"] = "Rotated + Noise"
df.loc[df["Representations"] == "shuffled", "Representations"] = "Shuffled Baseline"


In [ ]:
g = sns.relplot(
    # data=df[~df["dist"].isin(["squared_shuffled", "squared_ortho_proc", "shuffled"])],
    data=df[~df["dist"].isin(["squared_shuffled", "squared_ortho_proc"])],
    # data=df,
    x="D",
    y="value",
    hue="Representations",
    style="Noise SD",
    kind="line",
    markers=True,
    height=4,
    aspect=1.3,
    errorbar="sd",
)
g.set(yscale="log")
g.set(xscale="log")
g.set(ylabel="Orthogonal Procrustes Distance")
g.set(xlabel="Dimensionality D")

g.savefig("orthoproc_dim.pdf")